In [13]:
import torch
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup


def extract_features(url):
    parsed_url = urlparse(url)
    
    # TLD
    tld = parsed_url.netloc.split('.')[-1]
    
    # Feature hashing cho TLD
    tld_hashed = feature_hash(tld)
    
    # URL Length
    url_length = len(url)
    
    # Is Domain IP
    is_domain_ip = 1 if parsed_url.netloc.replace('.', '').isdigit() else 0
    
    # Number of Subdomains
    subdomains = parsed_url.netloc.split('.')
    num_subdomains = len(subdomains)-2
    
    # Number of Obfuscated Characters
    num_obfuscated_chars = sum([url.count(char) for char in ['@', '-', '_', '~']])
    
    # Is HTTPS
    is_https = 1 if parsed_url.scheme == 'https' else 0
    
    # Number of Digits in URL
    num_digits = sum(c.isdigit() for c in url)
    
    # Number of Equals in URL
    num_equals = url.count('=')
    
    # Number of Question Marks in URL
    num_qmark = url.count('?')
    
    # Number of Ampersands in URL
    num_ampersand = url.count('&')
    
    return {
        'TLD': tld_hashed,
        'URLLength': url_length,
        'IsDomainIP': is_domain_ip,
        'NoOfSubDomain': num_subdomains,
        'NoOfObfuscatedChar': num_obfuscated_chars,
        'IsHTTPS': is_https,
        'NoOfDegitsInURL': num_digits,
        'NoOfEqualsInURL': num_equals,
        'NoOfQMarkInURL': num_qmark,
        'NoOfAmpersandInURL': num_ampersand
    }

def feature_hash(value, num_features=100):
    # Sử dụng hàm hash để ánh xạ giá trị vào một phạm vi num_features
    return hash(value) % num_features
# Example URL to test
test_url = "https://www.arara.org"

# Extract features from the test URL
extract_features(test_url)

def extract_hyperlinks(url):
    # Tạo một danh sách trống để lưu trữ các liên kết
    hyperlinks = []
    
    try:
        # Gửi yêu cầu HTTP GET đến URL
        response = requests.get(url)
        
        # Nếu yêu cầu thành công, phân tích nội dung HTML
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Tìm tất cả các thẻ <a> trong nội dung HTML
            for link in soup.find_all('a'):
                href = link.get('href')
                
                # Nếu liên kết không rỗng, thêm vào danh sách liên kết
                if href:
                    hyperlinks.append(href)
    except Exception as e:
        print("Error extracting hyperlinks:", e)
    
    return hyperlinks


# Example URL

extract_hyperlinks ('https://www.arara.org')

{'TLD': 4,
 'URLLength': 21,
 'IsDomainIP': 0,
 'NoOfSubDomain': 1,
 'NoOfObfuscatedChar': 0,
 'IsHTTPS': 1,
 'NoOfDegitsInURL': 0,
 'NoOfEqualsInURL': 0,
 'NoOfQMarkInURL': 0,
 'NoOfAmpersandInURL': 0}

In [25]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_hyperlinks(url):
    # Sử dụng requests để lấy nội dung của trang web
    response = requests.get(url)
    
    # Kiểm tra xem request có thành công không
    if response.status_code != 200:
        return []  # Trả về một danh sách rỗng nếu request không thành công
    
    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Tìm tất cả các thẻ <a> trong trang web
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:  # Đảm bảo thuộc tính href tồn tại
            # Đưa đường dẫn tương đối thành đường dẫn tuyệt đối
            absolute_url = urljoin(url, href)
            # Thêm liên kết vào danh sách nếu nó không trùng lặp
            if absolute_url not in links:
                links.append(absolute_url)
    
    return links
extract_hyperlinks ('https://www.nottinghamshirewildlife.org')

['https://www.nottinghamshirewildlife.org#main-content',
 'https://www.nottinghamshirewildlife.org/',
 'https://www.nottinghamshirewildlife.org/what-we-do',
 'https://www.nottinghamshirewildlife.org/what-we-do/about-us',
 'https://www.nottinghamshirewildlife.org/what-we-do/about-us/who-we-are',
 'https://www.nottinghamshirewildlife.org/what-we-do/about-us/our-vision',
 'https://www.nottinghamshirewildlife.org/wildernottinghamshire',
 'https://www.nottinghamshirewildlife.org/our-vision-attenborough-nature-reserve',
 'https://www.nottinghamshirewildlife.org/our-vision-idle-valley-nature-reserve',
 'https://www.nottinghamshirewildlife.org/transform-our-towns',
 'https://www.nottinghamshirewildlife.org/what-we-do/about-us/our-history',
 'https://www.nottinghamshirewildlife.org/60th-anniversary',
 'https://www.nottinghamshirewildlife.org/60th-anniversary-views',
 'https://www.nottinghamshirewildlife.org/contactus',
 'https://www.nottinghamshirewildlife.org/what-we-do/about-us/fundraising-pr

In [24]:
import torch
from torch_geometric.data import Data

def url_to_graph(url):
    features = extract_features(url)
    
    # Tạo danh sách các nút và các đặc trưng tương ứng
    nodes = [url]
    node_features = [list(features.values())]
    
    # Trích xuất liên kết từ URL và thêm vào danh sách các nút và đặc trưng
    hyperlinks = extract_hyperlinks(url)
    for link in hyperlinks:
        nodes.append(link)
        features_link = extract_features(link)
        node_features.append(list(features_link.values()))
    
    # Tạo tensor chứa các đặc trưng của các nút
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Chuyển đổi cạnh thành danh sách cạnh (dạng (src, dst))
    edge_index = []
    
    # Thêm các cạnh từ URL gốc đến các liên kết mới
    url_index = nodes.index(url)
    for i, link in enumerate(hyperlinks):
        link_index = nodes.index(link)
        edge_index.append([url_index, link_index])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Tạo đối tượng Data từ các đặc trưng và cạnh
    data = Data(x=x, edge_index=edge_index)

    return data

# Example URL
url = "https://www.nottinghamshirewildlife.org"

# Convert URL to Graph
graph_data = url_to_graph(url)

# Print graph data
print("Graph Data:")
print(graph_data)

# Nodes
print("Nodes:")
for i, node in enumerate(graph_data.x):
    print(f"Node {i}: {node}")

# Cạnh
print("\nEdges:")
for src, dst in graph_data.edge_index.t().tolist():
    print(f"Edge: ({src}, {dst})")

# Đặc trưng của mỗi node
print("\nNode Features:")
for i, node_feature in enumerate(graph_data.x):
    print(f"Node {i} Features:")
    for j, feature_value in enumerate(node_feature):
        print(f"Feature {j}: {feature_value}")


Graph Data:
Data(x=[125, 10], edge_index=[2, 124])
Nodes:
Node 0: tensor([ 4., 39.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.])
Node 1: tensor([ 4., 52.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.])
Node 2: tensor([ 4., 40.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.])
Node 3: tensor([ 4., 50.,  0.,  1.,  2.,  1.,  0.,  0.,  0.,  0.])
Node 4: tensor([ 4., 59.,  0.,  1.,  3.,  1.,  0.,  0.,  0.,  0.])
Node 5: tensor([ 4., 70.,  0.,  1.,  5.,  1.,  0.,  0.,  0.,  0.])
Node 6: tensor([ 4., 70.,  0.,  1.,  4.,  1.,  0.,  0.,  0.,  0.])
Node 7: tensor([ 4., 61.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.])
Node 8: tensor([ 4., 78.,  0.,  1.,  4.,  1.,  0.,  0.,  0.,  0.])
Node 9: tensor([ 4., 77.,  0.,  1.,  5.,  1.,  0.,  0.,  0.,  0.])
Node 10: tensor([ 4., 59.,  0.,  1.,  2.,  1.,  0.,  0.,  0.,  0.])
Node 11: tensor([ 4., 71.,  0.,  1.,  4.,  1.,  0.,  0.,  0.,  0.])
Node 12: tensor([ 4., 56.,  0.,  1.,  1.,  1.,  2.,  0.,  0.,  0.])
Node 13: tensor([ 4., 62.,  0.,  1.,  2.,  1.,  2.,  0.,  0.,  0.])


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from torch_scatter import scatter_max


# Hàm extract_features và extract_hyperlinks đã được định nghĩa trước đó
def global_max_pool(x, batch):
    # Tính max pooling trên các nút trong từng batch
    return scatter_max(x, batch, dim=0)[0]

# Hàm tạo đồ thị từ URL
def url_to_graph_data(url, label):
    # Biểu diễn URL thành đồ thị
    graph_data = url_to_graph(url)
    
    # Tạo dữ liệu PyTorch Geometric từ đồ thị và nhãn
    data = {
        'x': graph_data.x,
        'edge_index': graph_data.edge_index,
        'y': torch.tensor(label, dtype=torch.long)
    }
    
    return data

# Load dataset từ file hoặc database
# Assume dataset là một danh sách các tuples (url, label)
df = pd.read_csv('/Users/rollie/Documents/URL_Phishing_Detection/notebooks/filtered_file.csv')
dataset = df[['URL', 'label']]

# Chia dataset thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)


# Kiểm tra độ dài của mỗi phần tử trong train_data
for data in train_data:
    if len(data) != 2:
        print("Error: Each element in train_data should be a tuple with exactly 2 values (URL and label).")
        break
else:
    # Nếu không có lỗi, tiếp tục biểu diễn dữ liệu thành đồ thị và tạo DataLoader
    train_graph_data = [url_to_graph_data(URL, label) for URL, label in train_data]
    train_loader = DataLoader(train_graph_data, batch_size=64, shuffle=True)


# Biểu diễn mỗi URL trong tập huấn luyện thành đồ thị và tạo DataLoader cho tập huấn luyện
train_graph_data = [url_to_graph_data(URL, label) for URL, label in train_data]
train_loader = DataLoader(train_graph_data, batch_size=64, shuffle=True)

# Biểu diễn mỗi URL trong tập kiểm tra thành đồ thị và tạo DataLoader cho tập kiểm tra
test_graph_data = [url_to_graph_data(URL, label) for URL, label in test_data]
test_loader = DataLoader(test_graph_data, batch_size=64, shuffle=False)

# Định nghĩa mô hình GNN
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = ...  # Định nghĩa lớp Convolutional Graph Neural Network
        self.conv2 = ...  # Định nghĩa lớp Convolutional Graph Neural Network
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = global_max_pool(x, data.batch)  # Global max pooling
        x = self.fc(x)
        return x

# Khởi tạo mô hình
model = GNNModel(input_dim=11, hidden_dim=64, output_dim=2)  # Input_dim là số lượng đặc trưng, ở đây là 11

# Định nghĩa hàm loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
for epoch in range(10):  # Ví dụ cho 10 epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch.y)
        loss.backward()
        optimizer.step()

# Đánh giá mô hình trên tập kiểm tra
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        output = model(batch)
        _, predicted = torch.max(output.data, 1)
        total += batch.y.size(0)
        correct += (predicted == batch.y).sum().item()

    print('Accuracy on test set: {:.2f}%'.format(100 * correct / total))


ValueError: too many values to unpack (expected 2)

In [26]:
pip install torch-scatter


18201.41s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 809.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp38-cp38-macosx_11_0_arm64.whl size=273724 sha256=c72dd427ce2da4629ff5241be791bff1c4934b0d6de850daf4a3d648bcb2c4aa
  Stored in directory: /Users/rollie/Library/Caches/pip/wheels/54/0c/3d/5e4aea36abfd59a43f8bb9859f545baa46c2a880a99a428db6
Successfully built torch-scatter
Note: you may need to restart the kernel to use updated packages.
